In [161]:
import numpy as np


# DSB Raw

In [162]:
import numpy as np

def discrete_simulated_bifurcation(Q, num_steps=100, dt=0.01):
    """
    Implements the Discrete Simulated Bifurcation algorithm to solve QUBO problems.
    
    Parameters:
    - Q: numpy array, the QUBO matrix (symmetric)
    - num_steps: int, number of simulation steps
    - dt: float, time step for updates
    
    Returns:
    - best_solution: binary numpy array of {-1,1}, representing the best found solution
    - best_energy: float, the lowest energy found
    """
    n = Q.shape[0]
    
    # Initialize variables
    s = np.random.choice([-1, 1], size=n)  # Spin states
    p = np.zeros(n)  # Momentum-like variables
    
    best_solution = s.copy()
    best_energy = s @ Q @ s

    # Time evolution
    for _ in range(num_steps):
        # Momentum update: p_i = p_i - dt * sum_j (Q_ij * s_j)
        p -= dt * (Q @ s)
        
        # Discrete bifurcation step
        s = np.sign(p)

        # Compute current energy
        current_energy = s @ Q @ s
        if current_energy < best_energy:
            best_solution = s.copy()
            best_energy = current_energy

    return best_solution, best_energy

# Example Usage
if __name__ == "__main__":
    # Define a small QUBO problem (4 variables)
    Q = np.array([
        [ 3, -2, -1,  0],
        [-2,  3, -2, -1],
        [-1, -2,  3, -2],
        [ 0, -1, -2,  3]
    ])

    solution, energy = discrete_simulated_bifurcation(Q)
    print("Best Solution:", solution)
    print("Best Energy:", energy)


Best Solution: [-1 -1 -1 -1]
Best Energy: -4


# DSB Explained

In [163]:

def discrete_simulated_bifurcation(Q, num_steps=100, dt=0.01):
    """
    Implements the Discrete Simulated Bifurcation algorithm to solve QUBO problems.
    
    This algorithm is inspired by quantum bifurcation machines and provides an efficient
    heuristic approach for finding good solutions to combinatorial optimization problems.
    
    Parameters:
    - Q: numpy array, the QUBO matrix (symmetric)
        This matrix represents the quadratic terms in the objective function we're minimizing.
        Q_ij represents the interaction strength between variables i and j.
    - num_steps: int, number of simulation steps
        Controls how long we run the simulation; more steps can find better solutions
        but increases computation time.
    - dt: float, time step for updates
        Controls the size of each momentum update; smaller values lead to more
        precise but slower evolution of the system.
    
    Returns:
    - best_solution: binary numpy array of {-1,1}, representing the best found solution
        Each element corresponds to a binary variable in the original problem.
    - best_energy: float, the lowest energy found
        The objective function value of the best solution.
    """
    # Get the problem size (number of variables)
    n = Q.shape[0]
    
    # Initialize variables
    s = np.random.choice([-1, 1], size=n)  # Spin states (Ising model representation)
                                           # We randomly initialize each variable to either -1 or 1
    
    p = np.zeros(n)  # Momentum-like variables
                     # These act like physical momentum, guiding the evolution of spin states
                     # All momenta start at zero
    
    # Keep track of the best solution found so far
    best_solution = s.copy()  # Store initial configuration as our first "best" solution
    best_energy = s @ Q @ s   # Calculate energy of initial state: E = s^T * Q * s
                              # This quadratic form gives us the objective function value
    
    # Time evolution - iterate through simulation steps
    for _ in range(num_steps):
        # Momentum update: p_i = p_i - dt * sum_j (Q_ij * s_j)
        # This is analogous to force = -gradient(energy)
        # Q @ s calculates the "effective field" acting on each spin
        p -= dt * (Q @ s)
        
        # Discrete bifurcation step
        # The sign function converts any positive momentum to +1 and negative to -1
        # This creates the "bifurcation" effect that helps escape local minima
        s = np.sign(p)

        # Compute current energy of the new spin configuration
        current_energy = s @ Q @ s  # Calculate E = s^T * Q * s for current state
        
        # If we found a better solution, update our records
        if current_energy < best_energy:
            best_solution = s.copy()  # Store this better configuration
            best_energy = current_energy  # Update the lowest energy found

    # Return the best solution found during the entire simulation
    return best_solution, best_energy

# Example Usage
if __name__ == "__main__":
    # Define a small QUBO problem (4 variables)
    Q = np.array([
        [0, 1, 1, 0, 1],
        [1, 0, 0, 1, 0],
        [1, 0, 0, 1, 1],
        [0, 1, 1, 0, 1],
        [1, 0, 1, 1, 0]
    ])

    solution, energy = discrete_simulated_bifurcation(Q)
    print("Best Solution:", solution)
    print("Best Energy:", energy)


Best Solution: [-1.  1.  1. -1.  1.]
Best Energy: -10.0


# DSB Updated

In [164]:

def discrete_simulated_bifurcation(Q, num_steps=100, dt=0.01):
    """
    Implements the Discrete Simulated Bifurcation algorithm to solve QUBO problems.
    
    This algorithm is inspired by quantum bifurcation machines and provides an efficient
    heuristic approach for finding good solutions to combinatorial optimization problems.
    
    Parameters:
    - Q: numpy array, the QUBO matrix (symmetric)
        This matrix represents the quadratic terms in the objective function we're minimizing.
        Q_ij represents the interaction strength between variables i and j.
    - num_steps: int, number of simulation steps
        Controls how long we run the simulation; more steps can find better solutions
        but increases computation time.
    - dt: float, time step for updates
        Controls the size of each momentum update; smaller values lead to more
        precise but slower evolution of the system.
    
    Returns:
    - best_solution: binary numpy array of {-1,1}, representing the best found solution
        Each element corresponds to a binary variable in the original problem.
    - best_energy: float, the lowest energy found
        The objective function value of the best solution.
    """
    # Get the problem size (number of variables)
    n = Q.shape[0]
    
    # Initialize variables
    s = 0.02 * (np.random.rand(n) - 0.5)  # Spin states (Ising model representation)
                                           # We randomly initialize each variable to either -1 or 1
    
    p = 0.02 * (np.random.rand(n) - 0.5)  # Momentum-like variables
                     # These act like physical momentum, guiding the evolution of spin states
                     # All momenta start at zero
    
    # Keep track of the best solution found so far
    best_solution = s.copy()  # Store initial configuration as our first "best" solution
    best_energy = s @ Q @ s   # Calculate energy of initial state: E = s^T * Q * s
                              # This quadratic form gives us the objective function value
    
    # Time evolution - iterate through simulation steps
    for _ in range(num_steps):
        # Momentum update: p_i = p_i - dt * sum_j (Q_ij * s_j)
        # This is analogous to force = -gradient(energy)
        # Q @ s calculates the "effective field" acting on each spin
        p -= dt * (Q @ s)
        
        # Discrete bifurcation step
        # The sign function converts any positive momentum to +1 and negative to -1
        # This creates the "bifurcation" effect that helps escape local minima
        s = np.sign(p)

        # Compute current energy of the new spin configuration
        current_energy = s @ Q @ s  # Calculate E = s^T * Q * s for current state
        
        # If we found a better solution, update our records
        if current_energy < best_energy:
            best_solution = s.copy()  # Store this better configuration
            best_energy = current_energy  # Update the lowest energy found

    # Return the best solution found during the entire simulation
    return best_solution, best_energy

# Example Usage
if __name__ == "__main__":
    # Define a small QUBO problem (4 variables)
    Q = np.array([
        [0, 1, 1, 0, 1],
        [1, 0, 0, 1, 0],
        [1, 0, 0, 1, 1],
        [0, 1, 1, 0, 1],
        [1, 0, 1, 1, 0]
    ])

    solution, energy = discrete_simulated_bifurcation(Q)
    print("Best Solution:", solution)
    print("Best Energy:", energy)


Best Solution: [-1.  1.  1. -1.  1.]
Best Energy: -10.0


# BSB Raw

In [165]:
import numpy as np

def ballistic_simulated_bifurcation(Q, num_steps=100, dt=0.01):
    """
    Implements the Ballistic Simulated Bifurcation (BSB) algorithm to solve QUBO problems.
    
    Parameters:
    - Q: numpy array, the QUBO matrix (symmetric)
    - num_steps: int, number of simulation steps
    - dt: float, time step for updates
    
    Returns:
    - best_solution: binary numpy array of {-1,1}, representing the best found solution
    - best_energy: float, the lowest energy found
    """
    n = Q.shape[0]
    
    # Initialize variables
    s = np.random.choice([-1, 1], size=n)  # Spin states
    p = np.zeros(n)  # Momentum-like variables
    
    best_solution = s.copy()
    best_energy = s @ Q @ s

    # Time evolution
    for _ in range(num_steps):
        # Ballistic update: p_i = p_i + dt * sum_j (Q_ij * s_j)
        p += dt * (Q @ s)  # <-- CHANGED: Ballistic update instead of dissipative
        
        # Bifurcation step: s_i = sign(p_i)
        s = np.sign(p)

        # Compute current energy
        current_energy = s @ Q @ s
        if current_energy < best_energy:
            best_solution = s.copy()
            best_energy = current_energy

    return best_solution, best_energy

# Example Usage
if __name__ == "__main__":
    Q = np.array([
        [0, 1, 1, 0, 1],
        [1, 0, 0, 1, 0],
        [1, 0, 0, 1, 1],
        [0, 1, 1, 0, 1],
        [1, 0, 1, 1, 0]
    ])
    solution, energy = ballistic_simulated_bifurcation(Q)
    print("Best Solution:", solution)
    print("Best Energy:", energy)


Best Solution: [1 1 1 1 1]
Best Energy: 14
